# Label side

In [1]:
import numpy as np
from jesse import helpers, research

_, trading_1m = research.get_candles(
    "Binance Perpetual Futures",
    "BTC-USDT",
    "1m",
    helpers.date_to_timestamp("2020-06-01"),
    helpers.date_to_timestamp("2025-04-01"),
    warmup_candles_num=0,
    caching=False,
    is_for_jesse=False,
)

trading_1m.shape

(2541600, 6)

In [2]:
np.save("data/btc_1m.npy", trading_1m)

In [3]:
import numpy as np

trading_1m = np.load("data/btc_1m.npy")
trading_1m.shape

(2541600, 6)

In [4]:
from custom_indicators.toolbox.bar.dollar_bar import estimate_dollar_bar_threshold

for i in [15, 30, 60, 150, 350]:
    threshold = estimate_dollar_bar_threshold(trading_1m, i)
    print(f"{i}m: {threshold}")

15m: 126948517.87795517
30m: 268405436.68481952
60m: 551319274.2985482
150m: 1421823390.033098
350m: 3307915640.791289


In [5]:
from jesse import helpers

from custom_indicators.toolbox.bar.dollar_bar import build_dollar_bar
from strategies.BinanceMLV2.config import DOLLAR_BAR_THRESHOLD_MID

print(DOLLAR_BAR_THRESHOLD_MID)

tp = helpers.date_to_timestamp("2025-03-25")

label_bar = build_dollar_bar(trading_1m, DOLLAR_BAR_THRESHOLD_MID)

print(label_bar.shape)

551319274.2985482
(42590, 6)


In [6]:
from custom_indicators.toolbox.labeler.zigzag_labeler import zigzag

res = zigzag(label_bar[:, 2], 0.015)
np.unique(res, return_counts=True)

(array([-1,  1]), array([20756, 21834]))

In [7]:
label = np.column_stack((label_bar, res))
label

array([[ 1.59097470e+12,  9.47927000e+03,  9.54201000e+03, ...,
         9.47812000e+03,  5.78140560e+04, -1.00000000e+00],
       [ 1.59100026e+12,  9.54201000e+03,  9.52399000e+03, ...,
         9.48689000e+03,  5.78233920e+04, -1.00000000e+00],
       [ 1.59102624e+12,  9.52400000e+03,  9.64894000e+03, ...,
         9.52028000e+03,  5.74046950e+04,  1.00000000e+00],
       ...,
       [ 1.74345084e+12,  8.24000000e+04,  8.23862000e+04, ...,
         8.23142000e+04,  6.71093600e+03, -1.00000000e+00],
       [ 1.74345462e+12,  8.23863000e+04,  8.22926000e+04, ...,
         8.22210000e+04,  6.69516200e+03, -1.00000000e+00],
       [ 1.74346080e+12,  8.22926000e+04,  8.25175000e+04, ...,
         8.22424000e+04,  6.72384300e+03, -1.00000000e+00]])

In [8]:
np.save("data/label_side.npy", label[label[:, 0] <= tp])
print(label.shape)
print(label[label[:, 0] <= tp].shape)

(42590, 7)
(42410, 7)


In [9]:
from jesse import utils

df_dollar_bar = utils.numpy_candles_to_dataframe(
    label_bar[label_bar[:, 0] <= tp], name_date="time"
)
df_dollar_bar["zigzag"] = res[label_bar[:, 0] <= tp]
df_dollar_bar.head(1)

,time,open,close,high,low,volume,zigzag
2020-06-01 01:25:00,2020-06-01 01:25:00,9479.27,9542.01,9615.0,9478.12,57814.056,-1


In [9]:
from lightweight_charts import Chart

chart = Chart(inner_width=1, inner_height=0.8)
chart2 = chart.create_subchart(position="bottom", width=1, height=0.2, sync=True)

chart.set(df_dollar_bar[["time", "open", "high", "low", "close", "volume"]])

line = chart2.create_line(name="zigzag", style="dotted")
line.set(df_dollar_bar[["time", "zigzag"]])

chart.show(block=True)

RuntimeError: asyncio.run() cannot be called from a running event loop

# Label meta

In [1]:
import lightgbm as lgb
import numpy as np
import pandas as pd

from model.config import SIDE_ALL

label_side_1h = np.load("data/label_side.npy")
print(label_side_1h.shape)
print(np.unique(label_side_1h[:, 6], return_counts=True))

df_features = pd.read_parquet("data/features_5h.parquet")
print(df_features.shape)

side_model = lgb.Booster(model_file="model/model_side.txt")
side_model_pred_prob = side_model.predict(df_features[SIDE_ALL])
df_features["side_model_res"] = side_model_pred_prob
side_model_pred = np.where(side_model_pred_prob > 0.5, 1, -1)
print(np.unique(side_model_pred, return_counts=True))

(9058, 7)
(array([-1.,  1.]), array([3646, 5412]))
(9058, 11841)
(array([-1,  1]), array([3554, 5504]))


In [2]:
meta_label = (side_model_pred.astype(int) == label_side_1h[:, 6].astype(int)).astype(
    int
)
np.unique(meta_label, return_counts=True)

(array([0, 1]), array([ 386, 8672]))

In [3]:
label_meta_1h = label_side_1h.copy()
label_meta_1h[:, 6] = meta_label
np.save("data/label_meta.npy", label_meta_1h)